### From Silver To Gold: Aggregation and KPI Tables

In [0]:
from pyspark.sql.types import StringType, IntegerType, DateType, BooleanType
import pyspark.sql.functions as F
from delta.tables import DeltaTable

#### Widgets

In [0]:
dbutils.widgets.text("catalog_name", "ecommerce", "Catalog Name")
dbutils.widgets.text("storage_account_name", "ecommcistorage05", "Storage Account Name")
dbutils.widgets.text("container_name", "ecomm-raw-data", "Container Name")

In [0]:
catalog_name = dbutils.widgets.get("catalog_name")
storage_account_name = dbutils.widgets.get("storage_account_name")
container_name = dbutils.widgets.get("container_name")

print(catalog_name, storage_account_name, container_name)

ecommerce stgcodebadlsdevus001 ecomm-raw-data


In [0]:
# readChangeFeed flag is used to read the change feed (_change_type column mainly)
df = spark.readStream \
.format("delta") \
.option("readChangeFeed", "true") \
.table(f"{catalog_name}.silver.slv_order_items")

df.limit(10).display()

dt,order_ts,customer_id,order_id,item_seq,product_id,quantity,unit_price_currency,unit_price,discount_pct,tax_amount,channel,coupon_code,_rescued_data,ingest_timestamp,source_file,processed_time,_change_type,_commit_version,_commit_timestamp
2025-08-30,2025-08-30T16:11:13.000Z,CUST000000041993,676466,3,2000000122779,1,INR,4492.0,0.0,540,Website,null,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z
2025-08-30,2025-08-30T16:45:04.000Z,CUST000000178008,676473,4,2000000264592,1,AUD,9.0,7.0,1,Website,null,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z
2025-08-30,2025-08-30T05:25:26.000Z,CUST000000212699,676542,1,2000000498683,1,INR,80646.0,20.0,7747,Mobile,new10,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z
2025-08-30,2025-08-30T20:53:21.000Z,CUST000000035766,676645,1,2000000081878,1,USD,11.0,8.0,2,Website,null,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z
2025-08-30,2025-08-30T06:57:23.000Z,CUST000000173569,676670,1,2000000108285,2,INR,2285.0,16.0,192,Mobile,null,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z
2025-08-30,2025-08-30T11:31:59.000Z,CUST000000083229,676754,3,2000000064680,1,GBP,6.0,0.0,1,Mobile,prime5,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z
2025-08-30,2025-08-30T06:13:51.000Z,CUST000000240361,676941,1,2000000094366,2,INR,2738.0,0.0,657,Mobile,null,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z
2025-08-30,2025-08-30T08:05:06.000Z,CUST000000285845,676955,1,2000000363806,1,AED,3049.0,7.0,341,Mobile,null,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z
2025-08-30,2025-08-30T04:52:41.000Z,CUST000000113426,676976,1,2000000229430,2,GBP,4.0,10.0,2,Mobile,null,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z
2025-08-30,2025-08-30T14:27:00.000Z,CUST000000103373,676996,1,2000000097527,4,CAD,89.0,19.0,15,Website,null,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z


In [0]:
df_union = df.filter("_change_type IN ('insert', 'update_postimage')")

In [0]:
df_union = df_union.withColumn(
    "gross_amount",
    F.col("quantity") * F.col("unit_price")
    )

# 2) Add discount_amount (discount_pct is already numeric, e.g., 21 -> 21%)
df_union = df_union.withColumn(
    "discount_amount",
    F.ceil(F.col("gross_amount") * (F.col("discount_pct") / 100.0))
)

# 3) Add sale_amount = gross - discount
df_union = df_union.withColumn(
    "sale_amount",
    F.col("gross_amount") - F.col("discount_amount") + F.col("tax_amount")
)

# add date id
df_union = df_union.withColumn("date_id", F.date_format(F.col("dt"), "yyyyMMdd").cast(IntegerType()))  # Create date_key

# Coupon flag
#  coupon flag = 1 if coupon_code is not null else 0
df_union = df_union.withColumn(
    "coupon_flag",
    F.when(F.col("coupon_code").isNotNull(), F.lit(1))
     .otherwise(F.lit(0))
)

df_union.limit(5).display()    

dt,order_ts,customer_id,order_id,item_seq,product_id,quantity,unit_price_currency,unit_price,discount_pct,tax_amount,channel,coupon_code,_rescued_data,ingest_timestamp,source_file,processed_time,_change_type,_commit_version,_commit_timestamp,gross_amount,discount_amount,sale_amount,date_id,coupon_flag
2025-08-30,2025-08-30T16:11:13.000Z,CUST000000041993,676466,3,2000000122779,1,INR,4492.0,0.0,540,Website,null,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z,4492.0,0,5032.0,20250830,0
2025-08-30,2025-08-30T16:45:04.000Z,CUST000000178008,676473,4,2000000264592,1,AUD,9.0,7.0,1,Website,null,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z,9.0,1,9.0,20250830,0
2025-08-30,2025-08-30T05:25:26.000Z,CUST000000212699,676542,1,2000000498683,1,INR,80646.0,20.0,7747,Mobile,new10,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z,80646.0,16130,72263.0,20250830,1
2025-08-30,2025-08-30T20:53:21.000Z,CUST000000035766,676645,1,2000000081878,1,USD,11.0,8.0,2,Website,null,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z,11.0,1,12.0,20250830,0
2025-08-30,2025-08-30T06:57:23.000Z,CUST000000173569,676670,1,2000000108285,2,INR,2285.0,16.0,192,Mobile,null,null,2025-10-14T17:10:28.503Z,abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/order_items/landing/order_items_2025-08-30.csv,2025-10-14T20:23:07.811Z,insert,1,2025-10-14T20:24:26.000Z,4570.0,732,4030.0,20250830,0


In [0]:
orders_gold_df = df_union.select(
    F.col("date_id"),
    F.col("dt").alias("transaction_date"),
    F.col("order_ts").alias("transaction_ts"),
    F.col("order_id").alias("transaction_id"),
    F.col("customer_id"),
    F.col("item_seq").alias("seq_no"),
    F.col("product_id"),
    F.col("channel"),
    F.col("coupon_code"),
    F.col("coupon_flag"),
    F.col("unit_price_currency"),
    F.col("quantity"),
    F.col("unit_price"),
    F.col("gross_amount"),
    F.col("discount_pct").alias("discount_percent"),
    F.col("discount_amount"),
    F.col("tax_amount"),
    F.col("sale_amount").alias("net_amount")
)

### Write to Gold Table

In [0]:
gold_checkpoint_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/checkpoint/gold/fact_order_items/"
print(gold_checkpoint_path)

def upsert_to_gold(microBatchDF, batchId):
    table_name = f"{catalog_name}.gold.gld_fact_order_items"
    if not spark.catalog.tableExists(table_name):
        print("creating new table")
        microBatchDF.write.format("delta").mode("overwrite").saveAsTable(table_name)
        spark.sql(
            f"ALTER TABLE {table_name} SET TBLPROPERTIES (delta.enableChangeDataFeed = true)"
        )
    else:
        deltaTable = DeltaTable.forName(spark, table_name)
        deltaTable.alias("gold_table").merge(
            microBatchDF.alias("batch_table"),
            "gold_table.transaction_id = batch_table.transaction_id AND gold_table.seq_no = batch_table.seq_no",
        ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

orders_gold_df.writeStream.trigger(availableNow=True).foreachBatch(
    upsert_to_gold
).format("delta").option("checkpointLocation", gold_checkpoint_path).option(
    "mergeSchema", "true"
).outputMode(
    "update"
).trigger(
    once=True
).start().awaitTermination()

abfss://ecomm-raw-data@stgcodebadlsdevus001.dfs.core.windows.net/checkpoint/gold/fact_order_items/


In [0]:
spark.sql(f"SELECT count(*) FROM {catalog_name}.gold.gld_fact_order_items").show()

+--------+
|count(*)|
+--------+
| 1191524|
+--------+



In [0]:
spark.sql(f"SELECT max(transaction_date) FROM {catalog_name}.gold.gld_fact_order_items").show()

+---------------------+
|max(transaction_date)|
+---------------------+
|           2025-09-02|
+---------------------+

